In [1]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB

In [2]:
class Instance:
    O: int  # number of requests
    I: int  # number of items
    A: int  # number of corridors

    u_oi: np.ndarray  # quantity of each item for each request
    u_ai = np.ndarray  # quantity of each item in each corridor

    LB: int  # lower bound on the wave size
    UB: int  # upper bound on the wave size

    def __init__(self, input_file):
        with open(input_file, 'r') as file:
            self.O, self.I, self.A = map(int, file.readline().split())

            # Read orders matrix:
            self.u_oi = np.zeros((self.O, self.I), dtype=int)
            for i in range(self.O):
                data = np.fromstring(file.readline(), dtype=int, sep=' ')
                indices, qtys = data[1::2], data[2::2]
                self.u_oi[i, indices] = qtys

            # Read corridors matrix:
            self.u_ai = np.zeros((self.A, self.I), dtype=int)
            for i in range(self.A):
                data = np.fromstring(file.readline(), dtype=int, sep=' ')
                indices, qtys = data[1::2], data[2::2]
                self.u_ai[i, indices] = qtys

            # Read lower and upper bounds for wave size:
            self.LB, self.UB = map(int, file.readline().split())


input_file = "datasets/a/instance_0010.txt"
inst = Instance(input_file)

# Nonlinear model

## Constants and variables:

In [3]:
# Create a new model
m = gp.Model("meli-nonlinear")

# Create variables
O_ = m.addMVar(inst.O, vtype=GRB.BINARY, name='o')  # wave selected requests
A_ = m.addMVar(inst.A, vtype=GRB.BINARY, name='a')  # wave visited corridors
obj = m.addVar(vtype=GRB.CONTINUOUS, lb=1, ub=inst.u_oi.sum(), name='Obj')  # the objective cost function

m.update()

Set parameter Username
Set parameter LicenseID to value 2615956
Academic license - for non-commercial use only - expires 2026-01-28


## Restrictions:

In [4]:
wave_size = (inst.u_oi.T @ O_).sum()  # total number of items in the wave

# Operational limits on the total number of items for the orders included in the wave:
m.addConstr(wave_size >= inst.LB)
m.addConstr(wave_size <= inst.UB)

# The selected corridors have sufficient storage for each of the items within the wave:
m.addConstrs(O_ @ inst.u_oi[:, i] <= A_ @ inst.u_ai[:, i] for i in range(inst.I))

m.addConstr(obj <= wave_size)  # basic cut
m.update()

## Objective function:

In [5]:
wave_corridors = A_.sum()  # number of used corridors
m.addConstr(obj == wave_size / wave_corridors)
m.setObjective(obj, GRB.MAXIMIZE)

## Model solving:

In [6]:
try:
    m.optimize()

    # Print the found solution:
    for v in m.getVars():
        print(f"{v.VarName} {v.X:g}")

    print(f"\nRequests, Items, Corridors: {inst.O}, {inst.I}, {inst.A}")
    print(f"Wave size: {wave_size.getValue():g}")
    print(f"Number of used corridors: {wave_corridors.getValue():g}")
except gp.GurobiError as e:
    print(f"Error code {e.errno}: {e}")
except AttributeError:
    print("Encountered an attribute error")

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Freedesktop SDK 23.08 (Flatpak runtime)")

CPU model: AMD Ryzen 7 8845HS w/ Radeon 780M Graphics, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 3692 rows, 1986 columns and 21471 nonzeros
Model fingerprint: 0x5696916b
Model has 1 general nonlinear constraint (1 nonlinear terms)
Variable types: 1 continuous, 1985 integer (1985 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 5e+03]
  RHS range        [4e+02, 2e+03]
Presolve model has 1 nlconstr
Added 2 variables to disaggregate expressions.
Presolve removed 126 rows and 22 columns
Presolve time: 0.05s
Presolved: 3572 rows, 1967 columns, 19514 nonzeros
Presolved model has 1 bilinear constraint(s)

Solving non-convex MIQCP

Variable types: 2 continuous, 1965 integer (1962 binary)

Root relaxation: objective 1.4

# Linearized model

In [7]:
# Create a new model
n = gp.Model("meli-linearized")

# Create variables
d = n.addVar(vtype=GRB.CONTINUOUS, lb=1 / inst.A, ub=1, name='d')  # the denominator of the objective cost function
θ = n.addMVar(inst.O, vtype=GRB.CONTINUOUS, lb=0, ub=1, name='θ')  # wave selected requests x d
α = n.addMVar(inst.A, vtype=GRB.CONTINUOUS, lb=0, ub=1, name='α')  # wave visited corridors x d

n.update()

## Restrictions and objective function:

In [8]:
wave_size_ = (inst.u_oi.T @ θ).sum()

# Operational limits on the total number of items for the orders included in the wave:
n.addConstr(wave_size_ >= inst.LB * d)
n.addConstr(wave_size_ <= inst.UB * d)

# The selected corridors have sufficient storage for each of the items within the wave:
n.addConstrs(θ @ inst.u_oi[:, i] <= α @ inst.u_ai[:, i] for i in range(inst.I))

wave_corridors_ = α.sum()  # number of used corridors x d
n.addConstr(wave_corridors_ == 1)  # equivalent to d = 1/wave_corridors_

n.update()

n.setObjective(wave_size_, GRB.MAXIMIZE)

## Solve linear relaxation:

In [9]:
dX = 1
try:
    n.optimize()

    # Print the found solution:
    dX = d.X
    for v in n.getVars():
        if v.VarName == 'd': continue
        print(f"{v.VarName} {v.X / dX:g}")
    print(f"Obj {wave_size_.getValue() / dX:g}")

    print(f"\nRequests, Items, Corridors: {inst.O}, {inst.I}, {inst.A}")
    print(f"Wave size: {wave_size_.getValue():g}")
    print(f"Number of used corridors: {wave_corridors_.getValue() / dX:g}")
except gp.GurobiError as e:
    print(f"Error code {e.errno}: {e}")
except AttributeError:
    print("Encountered an attribute error")

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Freedesktop SDK 23.08 (Flatpak runtime)")

CPU model: AMD Ryzen 7 8845HS w/ Radeon 780M Graphics, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 3692 rows, 1986 columns and 20253 nonzeros
Model fingerprint: 0x3ccfc545
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 2e+01]
  Bounds range     [3e-03, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 4 rows and 4 columns
Presolve time: 0.01s
Presolved: 3688 rows, 1982 columns, 20235 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.7460000e+03   1.090750e+03   0.000000e+00      0s
    4321    8.2643854e+01   0.000000e+00   0.000000e+00      0s

Solved in 4321 iterations and 0.26 seconds (0.54 work units)
Optimal objective  8.264385434e+01
θ[0] 1.25988
θ[1] 0
θ[2] 0
θ[3] 0.155201
θ[4] 0.0299475
θ[5] 0.00235

## Model solving:

In [10]:
# Create a new model
m = gp.Model("meli-nonlinear")

# Create variables
O_ = m.addMVar(inst.O, vtype=GRB.BINARY, name='o')  # wave selected requests
A_ = m.addMVar(inst.A, vtype=GRB.BINARY, name='a')  # wave visited corridors
obj = m.addVar(vtype=GRB.CONTINUOUS, lb=1, ub=inst.u_oi.sum(), name='Obj')  # the objective cost function

m.update()

wave_size = (inst.u_oi.T @ O_).sum()  # total number of items in the wave

# Operational limits on the total number of items for the orders included in the wave:
m.addConstr(wave_size >= inst.LB)
m.addConstr(wave_size <= inst.UB)

# The selected corridors have sufficient storage for each of the items within the wave:
m.addConstrs(O_ @ inst.u_oi[:, i] <= A_ @ inst.u_ai[:, i] for i in range(inst.I))

m.addConstr(obj <= wave_size)  # basic cut
m.addConstr(obj <= wave_size_.getValue() / dX)  # linear relaxation UB
m.update()

wave_corridors = A_.sum()  # number of used corridors
m.addConstr(obj == wave_size / wave_corridors)
m.setObjective(obj, GRB.MAXIMIZE)

try:
    m.optimize()

    # Print the found solution:
    for v in m.getVars():
        print(f"{v.VarName} {v.X:g}")

    print(f"\nRequests, Items, Corridors: {inst.O}, {inst.I}, {inst.A}")
    print(f"Wave size: {wave_size.getValue():g}")
    print(f"Number of used corridors: {wave_corridors.getValue():g}")
except gp.GurobiError as e:
    print(f"Error code {e.errno}: {e}")
except AttributeError:
    print("Encountered an attribute error")

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Freedesktop SDK 23.08 (Flatpak runtime)")

CPU model: AMD Ryzen 7 8845HS w/ Radeon 780M Graphics, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 3693 rows, 1986 columns and 21472 nonzeros
Model fingerprint: 0xfb2d2e24
Model has 1 general nonlinear constraint (1 nonlinear terms)
Variable types: 1 continuous, 1985 integer (1985 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 5e+03]
  RHS range        [4e+02, 2e+03]
Presolve model has 1 nlconstr
Added 2 variables to disaggregate expressions.
Presolve removed 127 rows and 23 columns
Presolve time: 0.03s
Presolved: 3572 rows, 1966 columns, 21090 nonzeros
Presolved model has 1 bilinear constraint(s)

Solving non-convex MIQCP

Variable types: 2 continuous, 1964 integer (1962 binary)

Interrupt request received

Ro